## Extração de dados do portal do INMET

Downlaod de arquivos zip do site (Github - https://dkko.me/posts/coleta-tratamento-inmet-bdmep/)

In [ ]:
import datetime as dt
import httpx
import datetime as dt

from pathlib import Path
from tqdm import tqdm
from pathlib import Path


def build_url(year):
    return f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}.zip'

def parse_last_modified(last_modified: str) -> dt.datetime:
    return dt.datetime.strptime(
    	last_modified,
    	'%a, %d %b %Y %H:%M:%S %Z'
    )

def build_local_filename(year: int, last_modified: dt.datetime) -> str:
    return f'inmet-bdmep_{year}_{last_modified:%Y%m%d}.zip'

def download_year(
    year: int,
    destdirpath: Path,
    blocksize: int = 2048,
) -> None:

    if not destdirpath.exists():
        destdirpath.mkdir(parents=True)

    url = build_url(year)

    headers = httpx.head(url).headers
    last_modified = parse_last_modified(headers['Last-Modified'])
    file_size = int(headers.get('Content-Length', 0))

    destfilename = build_local_filename(year, last_modified=last_modified)
    destfilepath = destdirpath / destfilename
    if destfilepath.exists():
        return

    with httpx.stream('GET', url) as r:
        pb = tqdm(
            desc=f'{year}',
            dynamic_ncols=True,
            leave=True,
            total=file_size,
            unit='iB',
            unit_scale=True,
        )
        with open(destfilepath, 'wb') as f:
            for data in r.iter_bytes(blocksize):
                f.write(data)
                pb.update(len(data))
        pb.close()

destdirpath = Path('C:/Users/z004wshk/OneDrive - Siemens Energy/Documents/BI Project')
for year in range(2000, dt.datetime.now().year + 1):
    download_year( year, destdirpath)

Coleta de dados dos arquivos ZIP

In [ ]:
import csv
import datetime as dt
import io
import re
import zipfile
import numpy as np
import pandas as pd
import datetime as dt

from pathlib import Path
from tqdm import tqdm
from pathlib import Path

def read_metadata(filepath: Path | zipfile.ZipExtFile) -> dict[str, str]:
    if isinstance(filepath, zipfile.ZipExtFile):
        f = io.TextIOWrapper(filepath, encoding='latin-1')
    else:
        f = open(filepath, 'r', encoding='latin-1')
    reader = csv.reader(f, delimiter=';')
    _, regiao = next(reader)
    _, uf = next(reader)
    _, estacao = next(reader)
    _, codigo_wmo = next(reader)
    _, latitude = next(reader)
    try:
        latitude = float(latitude.replace(',', '.'))
    except:
        latitude = np.nan
    _, longitude = next(reader)
    try:
        longitude = float(longitude.replace(',', '.'))
    except:
        longitude = np.nan
    _, altitude = next(reader)
    try:
        altitude = float(altitude.replace(',', '.'))
    except:
        altitude = np.nan
    _, data_fundacao = next(reader)
    if re.match('[0-9]{4}-[0-9]{2}-[0-9]{2}', data_fundacao):
        data_fundacao = dt.datetime.strptime(
            data_fundacao,
            '%Y-%m-%d',
        )
    elif re.match('[0-9]{2}/[0-9]{2}/[0-9]{2}', data_fundacao):
        data_fundacao = dt.datetime.strptime(
            data_fundacao,
            '%d/%m/%y',
        )
    f.close()
    return {
        'regiao': regiao,
        'uf': uf,
        'estacao': estacao,
        'codigo_wmo': codigo_wmo,
        'latitude': latitude,
        'longitude': longitude,
        'altitude': altitude,
        'data_fundacao': data_fundacao,
    }

def columns_renamer(name: str) -> str:
    name = name.lower()
    if re.match(r'data', name):
        return 'data'
    if re.match(r'hora', name):
        return 'hora'
    if re.match(r'precipita[çc][ãa]o', name):
        return 'precipitacao'
    if re.match(r'press[ãa]o atmosf[ée]rica ao n[íi]vel', name):
        return 'pressao_atmosferica'
    if re.match(r'press[ãa]o atmosf[ée]rica m[áa]x', name):
        return 'pressao_atmosferica_maxima'
    if re.match(r'press[ãa]o atmosf[ée]rica m[íi]n', name):
        return 'pressao_atmosferica_minima'
    if re.match(r'radia[çc][ãa]o', name):
        return 'radiacao'
    if re.match(r'temperatura do ar', name):
        return 'temperatura_ar'
    if re.match(r'temperatura do ponto de orvalho', name):
        return 'temperatura_orvalho'
    if re.match(r'temperatura m[áa]x', name):
        return 'temperatura_maxima'
    if re.match(r'temperatura m[íi]n', name):
        return 'temperatura_minima'
    if re.match(r'temperatura orvalho m[áa]x', name):
        return 'temperatura_orvalho_maxima'
    if re.match(r'temperatura orvalho m[íi]n', name):
        return 'temperatura_orvalho_minima'
    if re.match(r'umidade rel\. m[áa]x', name):
        return 'umidade_relativa_maxima'
    if re.match(r'umidade rel\. m[íi]n', name):
        return 'umidade_relativa_minima'
    if re.match(r'umidade relativa do ar', name):
        return 'umidade_relativa'
    if re.match(r'vento, dire[çc][ãa]o', name):
        return 'vento_direcao'
    if re.match(r'vento, rajada', name):
        return 'vento_rajada'
    if re.match(r'vento, velocidade', name):
        return 'vento_velocidade'
    # Retorna o nome original se não houver match
    return name
    
def convert_dates(dates: pd.Series) -> pd.Series:  # Corrigido o tipo de retorno
    dates = dates.str.replace('/', '-')
    return dates


def convert_hours(hours: pd.Series) -> pd.Series:  # Corrigido o tipo de retorno

    def fix_hour_string(hour: str) -> str:
        if pd.isna(hour):  # Verifica se é NaN
            return hour
        if re.match(r'^\d{2}\:\d{2}$', hour):
            return hour
        else:
            return hour[:2] + ':00'

    hours = hours.apply(fix_hour_string)
    return hours

def read_data(filepath: zipfile.ZipExtFile) -> pd.DataFrame:  # Corrigido o tipo do parâmetro
    d = pd.read_csv(
        filepath,
        sep=';',
        decimal=',',
        na_values='-9999',
        encoding='latin-1',
        skiprows=8,
        usecols=range(19),
    )
    d = d.rename(columns=columns_renamer)

    # Remove empty rows
    empty_columns = [
        'precipitacao',
        'pressao_atmosferica',
        'pressao_atmosferica_maxima',
        'pressao_atmosferica_minima',
        'radiacao',
        'temperatura_ar',
        'temperatura_orvalho',
        'temperatura_maxima',
        'temperatura_minima',
        'temperatura_orvalho_maxima',
        'temperatura_orvalho_minima',
        'umidade_relativa_maxima',
        'umidade_relativa_minima',
        'umidade_relativa',
        'vento_direcao',
        'vento_rajada',
        'vento_velocidade',
    ]
    empty_rows = d[empty_columns].isnull().all(axis=1)
    d = d.loc[~empty_rows]

    # Corrigido: aplicar as conversões nas colunas corretas
    d['data'] = convert_dates(d['data'])
    d['hora'] = convert_hours(d['hora'])

    return d

def read_zipfiles(destdirpath: Path) -> pd.DataFrame:
    data = pd.DataFrame()
    
    zip_files = list(destdirpath.glob('*.zip'))
    
    if not zip_files:
        print(f'Nenhum arquivo .zip encontrado em {destdirpath}')
        return data
    
    print(f'Encontrados {len(zip_files)} arquivos ZIP para processar')
    i=0
    for filepath in zip_files:
        print(f'Processando arquivo: {filepath.name}')
        try:
            with zipfile.ZipFile(filepath) as z:
                all_files = [zf for zf in z.infolist() if not zf.is_dir()]
                
                if not all_files:
                    print(f'Nenhum arquivo encontrado em {filepath.name}')
                    continue
                
                print(f'Exemplos de arquivos em {filepath.name}: {[f.filename for f in all_files[:3]]}')
                
                data_files = [zf for zf in all_files if 
                             zf.filename.lower().endswith(('.csv', '.txt')) or 
                             (not '.' in zf.filename.split('/')[-1] and zf.file_size > 100)]
                
                if not data_files:
                    data_files = [zf for zf in all_files if zf.file_size > 100]
                    print(f'Tentando processar {len(data_files)} arquivos de dados em {filepath.name}')
                else:
                    print(f'Encontrados {len(data_files)} arquivos de dados em {filepath.name}')
                
                for zf in tqdm(data_files, desc=f'Processando {filepath.name}'):
                    try:
                        meta = read_metadata(z.open(zf.filename))

                        if meta['uf']!='SP':
                            continue

                        d = read_data(z.open(zf.filename))
                        
                        if d.empty:
                            continue
                        
                        d = d.assign(**meta)
                        data = pd.concat([data, d], ignore_index=True)
                        
                    except Exception as e:
                        print(f'Erro ao processar arquivo {zf.filename}: {e}')
                        continue
                        
        except Exception as e:
            print(f'Erro ao abrir arquivo zip {filepath.name}: {e}')
            continue

    print(f'Processamento concluído. Total de registros: {len(data)}')
    
    if 'data' in data.columns and 'hora' in data.columns:
        try:
            data['datetime'] = pd.to_datetime(data['data'] + ' ' + data['hora'], errors='coerce')
        except:
            print('Não foi possível criar coluna datetime')
    
    return data

destdirpath = Path('C:/Users/z004wshk/OneDrive - Siemens Energy/Documents/BI Project')
df = read_zipfiles(destdirpath)

Encontrados 27 arquivos ZIP para processar
Processando arquivo: inmet-bdmep_2000_20200902.zip
Exemplos de arquivos em inmet-bdmep_2000_20200902.zip: ['2000/INMET_CO_DF_A001_BRASILIA_07-05-2000_A_31-12-2000.CSV', '2000/INMET_NE_BA_A401_SALVADOR_13-05-2000_A_31-12-2000.CSV', '2000/INMET_N_AM_A101_MANAUS_09-05-2000_A_31-12-2000.CSV']
Encontrados 5 arquivos de dados em inmet-bdmep_2000_20200902.zip


Processando inmet-bdmep_2000_20200902.zip: 100%|██████████| 5/5 [00:00<00:00, 1256.53it/s]

Processando arquivo: inmet-bdmep_2001_20200902.zip
Exemplos de arquivos em inmet-bdmep_2001_20200902.zip: ['2001/INMET_CO_DF_A001_BRASILIA_01-01-2001_A_31-12-2001.CSV', '2001/INMET_CO_GO_A002_GOIANIA_29-05-2001_A_31-12-2001.CSV', '2001/INMET_CO_GO_A003_MORRINHOS_25-05-2001_A_31-12-2001.CSV']
Encontrados 16 arquivos de dados em inmet-bdmep_2001_20200902.zip


Processando inmet-bdmep_2001_20200902.zip: 100%|██████████| 16/16 [00:00<00:00, 439.81it/s]


Processando arquivo: inmet-bdmep_2002_20200902.zip
Exemplos de arquivos em inmet-bdmep_2002_20200902.zip: ['2002/INMET_CO_DF_A001_BRASILIA_01-01-2002_A_31-12-2002.CSV', '2002/INMET_CO_GO_A002_GOIANIA_01-01-2002_A_31-12-2002.CSV', '2002/INMET_CO_GO_A003_MORRINHOS_01-01-2002_A_31-12-2002.CSV']
Encontrados 33 arquivos de dados em inmet-bdmep_2002_20200902.zip


...

Processando arquivo: inmet-bdmep_2025_20250902.zip
Exemplos de arquivos em inmet-bdmep_2025_20250902.zip: ['INMET_CO_DF_A001_BRASILIA_01-01-2025_A_31-08-2025.CSV', 'INMET_CO_DF_A042_BRAZLANDIA_01-01-2025_A_31-08-2025.CSV', 'INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2025_A_31-08-2025.CSV']
Encontrados 563 arquivos de dados em inmet-bdmep_2025_20250902.zip


Processando inmet-bdmep_2025_20250902.zip: 100%|██████████| 563/563 [00:13<00:00, 42.02it/s]  


Processando arquivo: inmet-bdmep_2025_20251104.zip
Exemplos de arquivos em inmet-bdmep_2025_20251104.zip: ['INMET_CO_DF_A001_BRASILIA_01-01-2025_A_31-10-2025.CSV', 'INMET_CO_DF_A042_BRAZLANDIA_01-01-2025_A_31-10-2025.CSV', 'INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2025_A_31-10-2025.CSV']
Encontrados 571 arquivos de dados em inmet-bdmep_2025_20251104.zip


Processando inmet-bdmep_2025_20251104.zip: 100%|██████████| 571/571 [00:13<00:00, 42.42it/s]  


Processamento concluído. Total de registros: 5359087


# Tratamento de dados metereológicos de algumas cidades de SP (dados tem limitações)

Configuração de datatype e agregação para análise por dia (média ou soma por dia)

In [ ]:
df['data']=df['data'].astype('datetime64[ns]')
df['regiao']=df['regiao'].astype('string')
df['uf']=df['uf'].astype('string')
df['estacao']=df['estacao'].astype('string')
df['codigo_wmo']=df['codigo_wmo'].astype('string')
df = (df.drop(columns=['hora', 'data_fundacao', 'datetime', 'pressao_atmosferica_maxima', 'pressao_atmosferica_minima',
                                'temperatura_orvalho_maxima', 'temperatura_orvalho_minima']))

agg_dict={}
for col in df.columns:
    if col=='regiao' or col=='uf' or col=='codigo_wmo':
        agg_dict[col]='unique'
    elif col=='precipitacao':
        agg_dict[col]='sum'
    elif col!='estacao' and col!='data':
        agg_dict[col]='mean'


df = df.groupby(['estacao','data']).agg(agg_dict)
df.reset_index().sort_values(['estacao', 'data'])

Implementação das métricas **Íncide de Calor** (Heat Index), **Sensação térmica** (Wind Chill Index - WCI) e **Índice de Umidade** (Humidex)

In [ ]:
def heat_index_celsius(T, R):
    # com T temperatura do ar e R umidade relativa
    c1 = -8.78469475556
    c2 = 1.61139411
    c3 = 2.33854883889
    c4 = -0.14611605
    c5 = -0.012308094
    c6 = -0.0164248277778
    c7 = 2.211732e-3
    c8 = 7.2546e-4
    c9 = -3.582e-6
    
    HI = (c1 + c2*T + c3*R + c4*T*R + c5*T**2 + c6*R**2 + 
          c7*T**2*R + c8*T*R**2 + c9*T**2*R**2)
    
    return HI
import math

def wci(V, T):
    # com V velocidade do vento e T temperatura do ar
    return (10 * math.sqrt(V) - V + 10.5) * (33 - T)

def humidex(T, T_o):
    # com T temperatura do ar e T_o temperatura de orvalho
    T_o_K = T_o + 273.15
    e = 6.11 * math.exp(5417.7530 * (1/273.16 - 1/T_o_K))
    humidex = T + 0.5555 * (e - 10.0)
    return humidex

df['Heat Index'] = df.apply(lambda x: heat_index_celsius(x['temperatura_ar'], x['umidade_relativa']), axis=1)
df['WCI'] = df.apply(lambda x: wci(x['vento_velocidade'], x['temperatura_ar']), axis=1)
df['Humidex'] = df.apply(lambda x: humidex(x['temperatura_ar'], x['temperatura_orvalho']),axis=1)

Download da base de dados para futuras análises

In [ ]:
df.to_excel('Data.xlsx', 'Database')